In [129]:
import cv2
import numpy as np

* Unlike earlier where we were detecting one color at a time, we now need to pick up on all the colors present in a given image.
* There are two ways to approach this. One we can go online and check for HSV values for different colors or we can use trackbars to manually find the HSV values for the different colors
* The following are values extracted for colours of our choice

### Function to detect color

Inputs
- input image 
- myColors : HSV min and max values for colours that we are interested in detecting . We obtain these values either manually using TradingBars or via interent
- myColorValues - Corresponding RGB values for the colors selected above

Functioning
- For each color in myColors, corresponding mask is calculated and is passed to the getContours function
- Color of the point is obtained through myColorValues and the coordinates and color info is passed on as a list of points (list of lists)

Output
- list of points with the following info (x,y,color_id)

In [153]:
def findColor(img,myColors,myColorValues):
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    count = 0
    newPoints = []
    for color in myColors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imgHSV,lower,upper)
        x,y = getContours(mask)
        cv2.circle(imgResult,(x,y),10,myColorValues[count],cv2.FILLED)
        if x!= 0 and y!=0:
            newPoints.append([x,y,count])
        count += 1
    return newPoints

### Function to detect contours

To see where the object is present within the frame

Inputs 
- image where the contours are to be found

Functioning
- Calculates area of contour, uses arclength to obtain boundary points and using this info approximation of the polygon is obtained using approxPolyDP()
- Bounding box coordinates are obtained for teh given polygon approximation using boundingRect()

Outputs
- We are interested in coordinates of the tip and not the bounding box . Hence the fnction returns the centre of top of bounding box

In [154]:
def getContours(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>500:
            #cv2.drawContours(imgResult, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx)
    return x+w//2,y

### Function to draw 

In [155]:
def drawOnCanvas(myPoints,myColorValues):
    for point in myPoints:
        cv2.circle(imgResult, (point[0], point[1]), 10, myColorValues[point[2]], cv2.FILLED)

In [158]:
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10,150)

myColors = [[5,107,0,19,255,255],
            [133,56,0,159,156,255],
            [57,76,0,100,255,255],
            [90,48,0,118,255,255]]

myColorValues = [[51,153,255],          ## BGR
                 [255,0,255],
                 [0,255,0],
                 [255,0,0]]
 
myPoints =  []  ## [x , y , colorId ]

while True:
    success, img = cap.read()
    imgResult = img.copy()
    newPoints = findColor(img, myColors,myColorValues)
    if len(newPoints)!=0:
        for newP in newPoints:
            myPoints.append(newP)
    if len(myPoints)!=0:
        drawOnCanvas(myPoints,myColorValues)
 
 
    cv2.imshow("Result", imgResult)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [159]:
cap.release()

In [150]:
cv2.destroyAllWindows()